# Web Scraping

In [ ]:
Name : DEVAKANTH KALAPATI

In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
import pandas as pd 
import time
import numpy as np
import os
from os.path import dirname, join

In [ ]:
#setting up the web driver
driver = webdriver.Chrome(executable_path=join(os.path.dirname("__file__"), 'chromedriver.exe'))
driver.implicitly_wait(60)
driver.get("https://www.bienici.com/recherche/achat/croix-59170,wasquehal-59290")


In [ ]:
#options_firefox = webdriver.FirefoxOptions()
#optionss.add_argument("-headless")
#optionss.headless = True
#options_firefox.set_preference("dom.push.enabled", False)
#driver = webdriver.Firefox(executable_path=r'C:\Users\dkalapati\Desktop\python individual\geckodriver.exe')

In [ ]:
#creating lists to store our scraped data

details_list = []
address_list = []
price_list = []
reference_list = []
img_list = []

In [ ]:
#using click method of selenium to go to a different page

driver.find_element_by_xpath('//*[@id="searchSideHeader"]/div/div[2]/div/div/div/div[2]').click()

In [ ]:
#getting the number of pages of data present for our search results

p = driver.find_elements_by_xpath('//*[@id="searchResultsContainer"]/div[3]/div/div[2]')
ph=[]
for i in p:
    s = i.text
    s = s.replace('\n'," ")
ph = s.split(" ")
for k in range(len(ph)):
    ph[k] = int(ph[k])
#del ph[0]
print(ph)




In [ ]:
#code block to get the data into the lists

#p = driver.find_elements_by_xpath('//*[@id="searchResultsContainer"]/div[3]/div/div[2]')

for page_num in ph:
    driver.implicitly_wait(60)

    #print("pagenumber ",i) 
    details = driver.find_elements_by_class_name('descriptionTitle')
    
    address = driver.find_elements_by_class_name('descriptionTitleAddress')
    
    price = driver.find_elements_by_class_name('thePrice')
    
    #price_per_sqm = driver.find_elements_by_class_name('searchItemPhoto')
    #reference_id = driver.find_elements_by_class_name('reference')
    
    img = driver.find_elements_by_class_name('searchItemPhoto')
    
    

    for a in details:
        details_list.append(a.text)
    
    for b in address:
        address_list.append(b.text)
        
    for c in price:
        if c.text != "":
            price_list.append(c.text)
    
    for p in img:
            img_list.append(p.get_attribute('src'))
            
    #for d in price_per_sqm:
         #if not any(g in d.text for g in data["price_per_sqm"]):
            #data["price_per_sqm"] = d.text
            
    #for e in reference_id:
        #reference_list.append(e.text)
        
    
        
    if page_num == ph[-1]:
        break
    else:
        pageiterator = f'//*[@id="searchResultsContainer"]/div[3]/div/div[2]/a[{page_num}]'
        time.sleep(10)
        driver.find_element_by_xpath(pageiterator).click()
        #pageiterator = f'https://www.bienici.com/recherche/achat/croix-59170,wasquehal-59290?page={page_num+1}'
        #driver.find_element_by_link_text(pageiterator).click()
        
    
    


In [ ]:
#closing the web driver
driver.quit()

In [ ]:
#converting the lists into dictionary
data_dict = {'category' : details_list,'address' : address_list, 'price' : price_list,'image_link' : img_list}

In [ ]:
data_dict

In [ ]:
#changing the default settings of column width of pandas dataframe to read in data of greater length

pd.options.display.max_colwidth = 250

In [ ]:
#converting the dictionary into dataframe
datah = pd.DataFrame(data_dict)

# cleaning data


In [ ]:
datah['price'] = datah['price'].str.replace(' ','')
#extracting number from price column
datah['price'] = datah['price'].str.extract('(\d+)', expand=False)
#converting price column to int
datah['price'] = datah['price'].transform(lambda h: int(h))

In [ ]:
datah['pincode'] = datah['address'].str.extract('(\d+)', expand=False)
datah['pincode'] = datah['pincode'].transform(lambda j: int(j))

In [ ]:
datah['city'] = datah['address'].transform(lambda t: t.str.split(' ',1).str[0])

In [ ]:
datah['street'] = datah['address'].transform(lambda t: t.str.split('(',1).str[1])
datah['street'] = datah['street'].str.replace(')','')

In [ ]:
datah['Type'] = datah['category'].transform(lambda t: t.str.split(' ',1).str[0])

In [ ]:
datah['nbr_rooms'] = datah['category'].str.extract('(\d+)', expand=False)
datah['nbr_rooms'] = datah['nbr_rooms'].transform(lambda t: float(t))

In [ ]:
datah['area'] = datah['category'].transform(lambda t: t.str.split('pièce').str[1])
datah['area'] = datah['area'].str.extract('(\d+)', expand=False)
datah['area'] = datah['area'].transform(lambda t: float(t))
datah['area'] = np.where(datah['area'] < 1000,datah['area'],np.nan)

In [ ]:
datah = datah[(datah['Type'] == "Maison") | (datah['Type'] =="Appartement")]

In [ ]:
#saving dataframe to csv file

datah.to_csv(join(os.path.dirname("__file__"), "ind_data_latest.csv"))